A current theory of mine is that certain parts of how this model breaks down inputs is through the mlps. I guess we can ablate MLPs to try to figure something out?

In [1]:
from transformer_lens import HookedTransformer
import torch
import matplotlib.pyplot as plt
import os
from jaxtyping import Float

torch.set_grad_enabled(False)

model = HookedTransformer.from_pretrained("gemma-2-2b", device="mps", dtype="float16")

/Users/sudarshanagopalkunnavakkam/Documents/Github/Language Models Updating Priors/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:16<00:00,  5.58s/it]


Loaded pretrained model gemma-2-2b into HookedTransformer


In [4]:
A = """
France, Paris
Japan, Tokyo
Brazil, Brasília
Canada, Ottawa
Australia, Canberra
Germany, Berlin
India, New_Delhi
Italy, Rome
Russia,"""

correct_answer = "87"
correct_answer_token = model.to_tokens(correct_answer, prepend_bos=False)

tokens = model.to_tokens(A)


In [6]:
def mlp_ablation_hook(mlp_out, hook):
    mlp_out[:, :, :] = 0


# model.blocks[2].hook_mlp_out.add_hook(mlp_ablation_hook)
# model(tokens)

for layer in range(model.cfg.n_layers):
    model.blocks[layer].hook_mlp_out.add_hook(mlp_ablation_hook)
    ablated_logits, ablated_cache = model.run_with_cache(tokens)
    
    print(ablated_logits.shape)
    topk = torch.topk(ablated_logits[0, -1, :], 5)
    out = model.tokenizer.convert_ids_to_tokens(topk.indices)

    model.reset_hooks()
    print(out)

torch.Size([1, 38, 256000])
['▁Moscow', 'Moscow', '▁Moskva', '▁Moscou', '▁Mos']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moskva', '▁Mos', '▁Moscou', '▁St']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moskva', '▁Mos', '▁St', '▁Moscou']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moscou', '▁Moskva', '▁Mos', '▁Moskau']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moskva', '▁Mos', '▁Moscou', '▁St']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moskva', '▁Moscou', '▁Mos', '▁St']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moskva', '▁Mos', '▁St', '▁Saint']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moskva', '▁Saint', '▁St', '▁Mos']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moskva', '▁Saint', '▁St', '▁Mos']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moskva', '▁Mos', 'Moscow', '▁St']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moskva', '▁Mos', '▁Moscou', '▁St']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moskva', '▁Mos', '▁St', '▁Moscou']
torch.Size([1, 38, 256000])
['▁Moscow', '▁Moskva', 'Moscow', '▁St', '▁Mos